In [ ]:
import monai
from monai.transforms import (
    AddChanneld,
    LoadImaged,
    ToTensord,
)
import torch as t
from copy import deepcopy
import torchio as tio
import numpy as np

Load images and copy for comparison

In [ ]:
path = "cropped_images/10_3T_nody_001.nii.gz"
add_channel = AddChanneld(keys=["image"])
loader = LoadImaged(keys=["image"])
to_tensor = ToTensord(keys=["image"])
monai_dict = {"image": path}
monai_image_layer = loader(monai_dict)
monai_image_layer = add_channel(monai_image_layer)
monai_image_layer = to_tensor(monai_image_layer)
monai_image_transform = deepcopy(monai_image_layer)
tio_copy = deepcopy(monai_image_layer)

Set up example affine

In [ ]:
translations = [0,0,0]
rotations = [t.pi/8, 0 ,0]

rotation_tensor = monai.transforms.utils.create_rotate(3, rotations,backend='torch')
translation_tensor = monai.transforms.utils.create_translate(3, translations,backend='torch')
affine =  (t.matmul(translation_tensor,rotation_tensor))

Define affine layer and transform

In [ ]:
monai_aff_layer = monai.networks.layers.AffineTransform(mode = "bilinear",  normalized = False, align_corners= True, padding_mode = "border")

s = t.diag(t.as_tensor([2 / (99 - 1.0), 2 / (92 - 1.0), 2 / (69 - 1.0), 1.0]))
aff_corrected = affine @ s
monai_aff_trans = monai.transforms.Affine(affine=aff_corrected, norm_coords = False, image_only = True)

Do transform using affine layer, add another dimension for batch

In [ ]:
input_batched = monai_image_layer["image"].unsqueeze(0)

monai_image_layer["image"] = monai_aff_layer(input_batched ,t.tensor(affine).unsqueeze(0))

monai_image_layer["image"] = monai_image_layer["image"].squeeze().unsqueeze(0)

nifti_saver = monai.data.NiftiSaver(output_dir="tests", output_postfix = "layer_non_norm",
                                            resample=False, padding_mode="zeros",
                                            separate_folder=False)
nifti_saver.save(monai_image_layer["image"],monai_image_layer["image_meta_dict"])

Do transform using transform.Affine

In [ ]:
monai_image_transform["image"] = monai_aff_trans(monai_image_transform["image"], padding_mode="border")

nifti_saver = monai.data.NiftiSaver(output_dir="tests", output_postfix = "trans_cor",
                                            resample=False, padding_mode="zeros",
                                            separate_folder=False)
nifti_saver.save(monai_image_transform["image"], monai_image_transform["image_meta_dict"])

In [ ]:
rotations = [t.pi/8, 0 ,0]

tio_image = tio.Image(tensor=tio_copy["image"])
tio_image.save('test/tio_init_eye.nii.gz')

rot_deg = np.array(rotations) * 180 / np.pi
tio_rotations = (rot_deg[0], rot_deg[1], rot_deg[2])

tio_aff = tio.Affine(1,tio_rotations,0,center="image",default_pad_value=0.0, image_interpolation="linear")


In [ ]:

tio_trans = tio_aff(tio_image)
tio_trans.save('tests/tio_lin_resample.nii.gz')


In [ ]:
tio_trans = tio_aff(tio_image)
#tio_trans = tio_resampler(tio_image)
tio_trans.save('tests/tio_lin_resample.nii.gz')

In [ ]:
a = t.tensor(affine).unsqueeze(0)

In [ ]:
a[0,:,:]

In [ ]:
affine